In [1]:
#!/usr/bin/env python
import cv2
import numpy as np
from video import create_capture
from common import clock, draw_str
from face_tracker import FaceTracker




In [2]:
import urllib 
import cv2
import numpy as np
import urllib.request

In [3]:
url="http://172.17.22.2:8080/shot.jpg?rnd=695158"
# while True:
#     imgResp=urllib.request.urlopen(url)
#     imgNp=np.array(bytearray(imgResp.read()),dtype=np.uint8)
#     img=cv2.imdecode(imgNp,-1)
#     cv2.imshow('test',img)
#     if ord('q')==cv2.waitKey(10):
#         exit(0)

In [4]:
face_cascade = cv2.CascadeClassifier("face.xml")
img=cv2.imread("grp4.jpeg")

count=0
while count<1:
    count=count+1
    #ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 3)
    print(faces)
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]

    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

#cap.release()
cv2.destroyAllWindows()
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[333 218  51  51]
 [703 251 180 180]
 [439 263 184 184]
 [ 80 286 276 276]]


In [5]:
yash_img=cv2.imread("yash.jpeg")
yagnesh_img=cv2.imread("yagnesh.jpeg")
kartikey_img=cv2.imread("kartikey.jpeg")
yash = cv2.cvtColor(yash_img, cv2.COLOR_BGR2GRAY)
yagnesh = cv2.cvtColor(yagnesh_img, cv2.COLOR_BGR2GRAY)
kartikey =cv2.cvtColor(kartikey_img, cv2.COLOR_BGR2GRAY)
yash = face_cascade.detectMultiScale(yash, 1.3, 3)
yagnesh = face_cascade.detectMultiScale(yagnesh, 1.3, 3)
kartikey = face_cascade.detectMultiScale(kartikey, 1.3, 3)

###########################################################
x=yash[0][0]
y=yash[0][1]
d=yash[0][2]
x1=x+d
y1=y+d
yash=cv2.resize(yash_img[y:y1,x:x1],(96,96))
############################################################
x=yagnesh[0][0]
y=yagnesh[0][1]
d=yagnesh[0][2]
x1=x+d
y1=y+d
yagnesh=cv2.resize(yagnesh_img[y:y1,x:x1],(96,96))
###########################################################
x=kartikey[0][0]
y=kartikey[0][1]
d=kartikey[0][2]
x1=x+d
y1=y+d
kartikey=cv2.resize(kartikey_img[y:y1,x:x1],(96,96))
##########################################################


In [6]:
cv2.imwrite("only_yash.jpg",yash);
cv2.imwrite("only_yagnesh.jpg",yagnesh);
cv2.imwrite("only_kartikey.jpg",kartikey);

In [7]:
#remove transpose before evaluating
# cv2.imshow('img',kartikey)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [8]:
n=faces.shape[0]

In [9]:
images={}
count=-1
for i in faces:
    count=count+1
    x=i[0]
    y=i[1]
    d=i[2]
    x1=x+d
    y1=y+d
    images[count]=cv2.resize(img[y:y1,x:x1],(96,96))
    cv2.imshow("imgage",images[count])
    cv2.imwrite(str(count)+".jpg",images[count]);
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

In [10]:

# grayscaled = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# th = cv2.adaptiveThreshold(grayscaled, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 115, 1)

In [11]:
# img2=cv2.resize(images[0],(96,96))

In [12]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [13]:

FRmodel = faceRecoModel(input_shape=(3, 96, 96))
print("Total Params:", FRmodel.count_params())

Instructions for updating:
Colocations handled automatically by placer.
Total Params: 3743280


In [14]:
# GRADED FUNCTION: triplet_loss

def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    ### START CODE HERE ### (≈ 4 lines)
    # Step 1: Compute the (encoding) distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist =tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    # Step 2: Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist),alpha)
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    ### END CODE HERE ###
    
    return loss

In [15]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [16]:
database = {}
#database["yash"] = img_to_encoding("only_yash.jpg", FRmodel)
#database["yagnesh"] = img_to_encoding("only_yagnesh.jpg", FRmodel)
database["kartikey"] = img_to_encoding("only_kartikey.jpg", FRmodel)

In [17]:
# GRADED FUNCTION: who_is_it

def who_is_it(image_path, database, model):
    """
    Implements face recognition for the happy house by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    
    ### START CODE HERE ### 
    
    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above. ## (≈ 1 line)
    encoding = img_to_encoding(image_path, FRmodel)
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = 1000
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current "emb" from the database. (≈ 1 line)
        dist = np.linalg.norm((encoding-db_enc))

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if (min_dist>dist):
            min_dist = dist
            identity = name

    ### END CODE HERE ###
    
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [18]:
who_is_it("0.jpg", database, FRmodel)

Not in the database.


(1.0852343, 'kartikey')

In [19]:
who_is_it("1.jpg", database, FRmodel)

Not in the database.


(0.8014409, 'kartikey')

In [20]:
who_is_it("2.jpg", database, FRmodel)

Not in the database.


(1.0024636, 'kartikey')

In [21]:
who_is_it("3.jpg", database, FRmodel)

Not in the database.


(0.998834, 'kartikey')